In [ ]:
# I don't think all of these are needed as this was part of a larger file when I got it.  However, I didn't have time to look
# through and remove the ones that are unnecessary.
import os
import urllib.request
import shutil
import re
from pathlib import Path

from os.path import expanduser
import pandas as pd
import glob
import io
from bs4 import BeautifulSoup

import sys
import unicodedata
import requests

In [ ]:
# Code to download index files. This code is from stackoverflow - https://stackoverflow.com/questions/68131406/downloading-files-from-sec-gov-via-edgar-using-python-3-9
# The key is to specify a header which signals to the SEC server that you are a real user and not a bot. Edit personal info
# below to include your company name and a real email address.

def download(year):
    for qtr in range(1, 5):
        url = f"https://www.sec.gov/Archives/edgar/full-index/{year}/QTR{qtr}/master.idx"
        response = requests.get(url, headers={'User-Agent': 'University of Southern California payneman@my.usc.edu'}, stream=True)
        print(url)
        response.raise_for_status()
        down_direct = os.path.join(Path.home(), 'Dropbox', 'Working_Papers', 'Lorien--Options', 'Coding_Files','Index')

        with open(f'{down_direct}/master{year}QTR{qtr}.idx', 'wb') as f:
            f.write(response.content)

#Specify start and end year of desired index
start_year =2006
end_year = 2013
for i in range(start_year,end_year+1):
    download(i)

In [ ]:
# From Bochkay textbook (12.2.2), modified to include User Agent header, which circumvents 403 Error Code
# Make sure to edit company name and email address in "response = request."" line of code

# Places in this cell to edit: 
# Bottom 4 lines of code --> Specify regex file you want (13F in this case), file locations, and years
# Can adjust quarters at "for qtr in range (1,5)"
# Can adjust "count" to test code before running it for full year

def get_files(start_year:int, end_year:int,
              reform:str, 
              inddirect:str, odirect:str):
    """
    Downloads SEC filings for specific companies
    start_year -> First Year to download
    end_year -> Last Year to download
    reform -> Regex to specify forms to be downloaded
    inddirect -> Directory containing index files
    odirect -> Directory the filings will be downloaded to
    """
    
    print('Downloading Filings')
    
    # Regex to identify the form to download.
    re_formtype = re.compile(reform, re.IGNORECASE)
    # Regex to extract file name information 
    # from a line
    re_fullfilename = re.compile(r"\|(edgar/data.*\/([\d-]+\.txt))", re.IGNORECASE)
    
    #loop through the index files based on year
    for year in range(start_year, end_year+1):
        #check whether the directory exists and create one if it does not.
        download_path = os.path.join(odirect, str(year))
        if not os.path.exists(download_path):
            os.makedirs(download_path)
                
        for qtr in range(1,5):
            #name of index file to be read.
            dl_file = os.path.join(inddirect, 'master' + str(year) + 'QTR' + str(qtr) + '.idx')
        
            # open the index file
            with open(dl_file, 'r') as f:
                # set a counter called count to 1. Note 
                # that the counter will only be incremented 
                # after it downloads a file.
                    count=1
                # loop through each line in the index file, assigning to a variable called line
                    for line in f:
                    # Only download a file if the counter is less than 5 (to test code). Remove this if statement if you want
                    # to download all the files for the time period or adjust if you want to expand sample.
                       # if count<5:
                        # Check to see if the line matches the form type 
                            rematch=re.search(re_formtype,line)
                            #If there is a match then download the filing
                            if rematch:
                                # The following line searches for filename information. 
                                # The first grouping will contain the location and filename of the file to be 
                                # downloaded. The second grouping will contain just contain the filename o
                                matches = re.search(re_fullfilename, line)
                                if matches:
                                    # Construct the url to for retrieving the filing 
                                    url = str('https://www.sec.gov/Archives/') + str(matches.group(1))
                                    # Create the filename to download the file to.
                                    outfile = os.path.join(download_path, 
                                                           str(matches.group(2)))
                                    # Check to make sure the file hasn't already been downloaded                    
                                    if not (os.path.isfile(outfile) and os.access(outfile, os.R_OK)):
                                        # Print the name of the 
                                        # file to be downloaded.
                                        print("Downloading:"+str(outfile),end='\n')
                                        response = requests.get(url, headers={'User-Agent': 'University of Southern California payneman@my.usc.edu'}, stream=True)
                                        response.raise_for_status()
                                        with open(f'{outfile}', 'wb') as f:
                                            f.write(response.content)   
                                        count += 1
    print('Downloading of Filings Complete',end='\n')
    return
                                                       
# Specify, in regular expression format, the filing
# you are looking for.  Following is the for 10-k.
# reform='(\|10-?k(sb|sb40|405)?\s*\|)'

# For 13F-HR filing
reform='(\|13f-?HR*\|)'

# Specify location of the index files.
inddirect = os.path.join(Path.home(), 'Dropbox', 'Working_Papers', 'Lorien--Options', 'Coding_Files','Index')

# Specify where to download filings to
odirect = os.path.join(Path.home(), 'Dropbox', 'Working_Papers', 'Lorien--Options', 'Coding_Files','13F')

# Execute the get filings function
get_files(2006, 2006, reform, inddirect, odirect)